In [1]:
import matplotlib.pyplot as plt
import nbfigtulz as ftl
import numpy as np

!mkdir -p build
ftl.config['img_dir'] = 'build'

In [2]:
def eval_pot(x, *, j, threshold=5):
    y = 1. / (2. * np.pi * j + x) + 1. / (2. * np.pi * (j + 1) - x)
    
    xs = []
    ys = []
    
    i_first = 0
    seq_done = False
    for i in range(len(x)):
        if abs(y[i]) > threshold:
            if not seq_done:
                seq_done = True
                xs.append(x[i_first:i])
                ys.append(y[i_first:i])
        elif seq_done:
            seq_done = False
            i_first = i
            
    xs.append(x[i_first:])
    ys.append(y[i_first:])
    
    return xs, ys


@ftl.with_context
def make_fig(n):
    x = np.linspace(-14, 20.5, 800)
    seqs = [eval_pot(x, j=j) for j in range(n)]
    
    fig, ax = plt.subplots()
    
    xticks = np.array([-4, -2, 0, 2, 4, 6]) * np.pi
    xticks_labels = [f'${x}$' for x in [r'-4\pi', r'-2\pi', r'0', r'2\pi', r'4\pi', r'6\pi']]
    ax.set_xticks(ticks=xticks, labels=xticks_labels)
    
    ax.set_xlabel(r'$\sigma$')
    ax.set_ylabel(r'$V_{\!3\mathrm{D}}^{(j)}(\sigma)$')
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(-4, 4)
    ax.grid()
    
    for j, seq in enumerate(seqs):
        label = f'$j={j}$'
        
        for x0, y0 in zip(*seq):
            s1 = (x0 > 0) 
            s2 = (x0 < 2. * np.pi) 
            ax.plot(x0[s1 & s2], y0[s1 & s2], color=f'C{j}', label=label)
            
            sel = (x0 < 0)
            ax.plot(x0[~s1], y0[~s1], '--', color=f'C{j}')
            
            sel = (x0 > 2. * np.pi)
            ax.plot(x0[~s2], y0[~s2], '--', color=f'C{j}')
            label = None
            
    ax.legend()
    
    return ftl.save_fig(fig, 'pot3D')


make_fig(3)

pot3D.png